In [4]:
import sys
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, PowerTransformer
from sklearn import set_config; set_config(display='diagram')

sys.path.append("/Users/antonis/code/Ant-mel/legendary_game_recs/")
from preprocessing.preprocess_1_cleaning import *
from preprocessing.preprocess_2_features import *
from preprocessing.pipeline_the_unification import *
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from model.model_create import create_baseline_model, predict_baseline_model

from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer
import datetime
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.neighbors import KNeighborsRegressor


In [2]:
def remove_link_trail(string):
    update = string.replace('https://www.backloggd.com', '')
    return update

In [3]:
api_cols_to_drop = ['game_engines', 'aggregated_rating', 'rating',
                    'aggregated_rating_count', 'age_ratings', 'multiplayer_modes']

In [4]:
api_1 = pd.read_json('../raw_data/api_json_0to30k')
api_2 = pd.read_json('../raw_data/api_json_30kto40k')
api_3 = pd.read_json('../raw_data/api_json_40kto50k')
api_4 = pd.read_json('../raw_data/api_json_50kto60k')
api_5 = pd.read_json('../raw_data/api_json_60kto90k')
api_6 = pd.read_json('../raw_data/api_json_90kto100k')
api_7 = pd.read_json('../raw_data/api_json_100kto110k')
api_8 = pd.read_json('../raw_data/api_json_110ktoEndk')

In [5]:
api_data = pd.concat((api_1, api_2, api_3, 
                      api_4, api_5, api_6, 
                      api_7, api_8), ignore_index=True)

In [6]:
api_cols_to_drop = ['game_engines', 'aggregated_rating', 'rating',
                    'aggregated_rating_count', 'age_ratings', 'multiplayer_modes']

extra_cols_to_drop = ['link', 'game_id', 'id', 'game_modes', 
                      'name', 'themes', 'player_perspectives']

In [7]:
def get_category_descriptions(json_dict):
    category_list = []
    
    if type(json_dict) == float:
        category_list = None
    else:
        for i in range(len(json_dict)):
            try:
                descriptions = json_dict[i]['content_descriptions']
                for i in range(len(descriptions)):
                    rating_descript = descriptions[i]['category']
                    category_list.append(rating_descript)
                    
            except:
                pass
    
    return category_list

def get_names_from_dict(dict_list):
    list_of_items = []
    try:
        for i in range(len(dict_list)):
            item = dict_list[i]['name']
            list_of_items.append(item)
    except:
        list_of_items = None
    
    return list_of_items

def prepare_json_df(original_json_df):
    
    original_json_df['themes'] = original_json_df['themes'].apply(get_names_from_dict)
    original_json_df['game_modes'] = original_json_df['game_modes'].apply(get_names_from_dict)
    original_json_df['player_perspectives'] = original_json_df['player_perspectives'].apply(get_names_from_dict)
    original_json_df['game_engines'] = original_json_df['game_engines'].apply(get_names_from_dict)
    original_json_df['age_ratings'] = original_json_df['age_ratings'].apply(get_category_descriptions)
    
    return original_json_df

In [8]:
def encode_api_data(raw_data, link_data, api_data):
    reference = pipeline_genre_ohe_only(raw_data)
    reference['url'] = reference['url'].apply(remove_link_trail)
    
    data_with_id = reference.merge(link_data, left_on='url', right_on='link')
    all_data = data_with_id.merge(api_data, left_on='game_id', right_on='id')
    
    prepped_data = prepare_json_df(all_data)    
    nec_data = prepped_data.drop(api_cols_to_drop, axis=1)
    filled_data = nec_data.fillna('')
    
    ohe_modes, mlb = categorical_encoder(filled_data['game_modes'])
    ohe_theme= keeping_ohe_columns_and_dropping_the_originals(filled_data[['themes']], 16)
    ohe_perspect, mlb = categorical_encoder(filled_data['player_perspectives'])
    
    ohe_colums = pd.concat((ohe_modes, ohe_theme, ohe_perspect), axis=1)
    
    everything = pd.concat((filled_data, ohe_colums), axis=1)
    
    return everything.drop(extra_cols_to_drop, axis=1)
    

In [9]:
num_transformer_yeo = PowerTransformer(method='yeo-johnson', standardize=False)

col_transformer = ColumnTransformer([('num_transformer', num_transformer_yeo,
                                   ['plays','playing','backlogs','wishlist','total_reviews','total_lists'])],
                                  remainder='passthrough')

def make_training_data_box_yeo(reference_data):
    dropped = drop_unnecesary_coulumns(reference_data)

    transformed = pd.DataFrame(col_transformer.fit_transform(dropped))

    X_train = transformed.drop(6, axis=1)
    y_train = transformed[6]

    return X_train, y_train

# Baseline model

In [2]:
game_data = pd.read_csv('../raw_data/all_game_data_v1_corrected2')
links_id = pd.read_csv('../raw_data/all_links_with_game_id_v1')

In [3]:
baseline_model, baseline_reference, X_train_baseline = create_baseline_model(game_data)

/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column]=pd.to_datetime(df[column])
/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_2_features.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupe['gen'] = None
/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [5]:
predict_baseline_model(11702, baseline_model, baseline_reference, X_train_baseline)

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,platforms,description,...,Simulator,Puzzle,Shooter,Strategy,Platform,Arcade,Sport,gen_1,gen_2,gen_3
11702,The Legend of Zelda: Breath of the Wild,2017-03-03,42000,3500,6600,3400,"[Nintendo EPD Production Group No. 3, Nintendo]",4.4,"[Wii U, Nintendo Switch]",The Legend of Zelda: Breath of the Wild is the...,...,0,1,0,0,0,0,0,0,0,1
2293,Elden Ring,2022-02-25,27000,4700,7300,6400,"[FromSoftware, Bandai Namco Entertainment]",4.5,"[Windows PC, PlayStation 4, Xbox One, PlayStat...","Elden Ring is a fantasy, action and open world...",...,0,0,0,0,0,0,0,0,0,1
15141,Bloodborne,2015-03-24,23000,1400,7900,4600,"[FromSoftware, Sony Computer Entertainment]",4.5,[PlayStation 4],An action RPG in which the player embodies a H...,...,0,0,0,0,0,0,0,0,0,1
15353,Life is Strange,2015-01-29,20000,285,2900,1100,"[Feral Interactive, ""DONT NOD]",3.5,"[Windows PC, Android, Mac, Xbox 360, Linux, Pl...",Life is Strange is a five part episodic game t...,...,0,1,0,0,0,0,0,0,0,1
13475,Dark Souls III,2016-03-24,26000,1200,6200,3100,"[Bandai Namco Entertainment, FromSoftware]",4.3,"[Windows PC, PlayStation 4, Xbox One]",Dark Souls continues to push the boundaries wi...,...,0,0,0,0,0,0,0,0,0,1
14883,The Witcher 3: Wild Hunt,2015-05-19,23000,1600,9000,2100,"[CD Projekt RED, Spike ChunSoft]",4.3,"[Windows PC, PlayStation 4, Xbox One, PlayStat...","RPG and sequel to The Witcher 2 (2011), The Wi...",...,0,0,0,0,0,0,0,0,0,1
274,The Legend of Zelda: Tears of the Kingdom,2023-05-12,11000,6200,4700,6200,"[Nintendo, Nintendo EPD Production Group No. 3]",4.5,[Nintendo Switch],The Legend of Zelda: Tears of the Kingdom is t...,...,0,0,0,0,0,0,0,0,0,1
14354,Undertale,2015-09-15,37000,886,7100,2500,"[tobyfox, 8-4]",4.3,"[Windows PC, Mac, Linux, PlayStation 4, Xbox O...","A small child falls into the Underground, wher...",...,0,0,0,0,0,0,0,0,0,1
2430,Pokémon Legends: Arceus,2022-01-28,13000,1800,3700,2600,"[Nintendo, Game Freak]",3.7,[Nintendo Switch],The Pokémon Legends: Arceus game honors past P...,...,0,0,0,0,0,0,0,0,0,1
12568,Persona 5,2016-09-15,20000,747,3500,2000,"[Deep Silver, P Studio]",4.1,"[PlayStation 4, PlayStation 3]","Persona 5, a turn-based JRPG with visual novel...",...,0,0,0,0,0,0,0,0,0,1


In [6]:
zelda = X_train_baseline[11702:11703]
wipeout = X_train_baseline[31357:31358]
omori = X_train_baseline[32334:32335]

In [7]:
ind_list = list(baseline_model.kneighbors(zelda,n_neighbors=11)[1][0])
baseline_reference.iloc[ind_list]

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,platforms,description,...,Simulator,Puzzle,Shooter,Strategy,Platform,Arcade,Sport,gen_1,gen_2,gen_3
11702,The Legend of Zelda: Breath of the Wild,2017-03-03,42000,3500,6600,3400,"[Nintendo EPD Production Group No. 3, Nintendo]",4.4,"[Wii U, Nintendo Switch]",The Legend of Zelda: Breath of the Wild is the...,...,0,1,0,0,0,0,0,0,0,1
2293,Elden Ring,2022-02-25,27000,4700,7300,6400,"[FromSoftware, Bandai Namco Entertainment]",4.5,"[Windows PC, PlayStation 4, Xbox One, PlayStat...","Elden Ring is a fantasy, action and open world...",...,0,0,0,0,0,0,0,0,0,1
15141,Bloodborne,2015-03-24,23000,1400,7900,4600,"[FromSoftware, Sony Computer Entertainment]",4.5,[PlayStation 4],An action RPG in which the player embodies a H...,...,0,0,0,0,0,0,0,0,0,1
15353,Life is Strange,2015-01-29,20000,285,2900,1100,"[Feral Interactive, ""DONT NOD]",3.5,"[Windows PC, Android, Mac, Xbox 360, Linux, Pl...",Life is Strange is a five part episodic game t...,...,0,1,0,0,0,0,0,0,0,1
13475,Dark Souls III,2016-03-24,26000,1200,6200,3100,"[Bandai Namco Entertainment, FromSoftware]",4.3,"[Windows PC, PlayStation 4, Xbox One]",Dark Souls continues to push the boundaries wi...,...,0,0,0,0,0,0,0,0,0,1
14883,The Witcher 3: Wild Hunt,2015-05-19,23000,1600,9000,2100,"[CD Projekt RED, Spike ChunSoft]",4.3,"[Windows PC, PlayStation 4, Xbox One, PlayStat...","RPG and sequel to The Witcher 2 (2011), The Wi...",...,0,0,0,0,0,0,0,0,0,1
274,The Legend of Zelda: Tears of the Kingdom,2023-05-12,11000,6200,4700,6200,"[Nintendo, Nintendo EPD Production Group No. 3]",4.5,[Nintendo Switch],The Legend of Zelda: Tears of the Kingdom is t...,...,0,0,0,0,0,0,0,0,0,1
14354,Undertale,2015-09-15,37000,886,7100,2500,"[tobyfox, 8-4]",4.3,"[Windows PC, Mac, Linux, PlayStation 4, Xbox O...","A small child falls into the Underground, wher...",...,0,0,0,0,0,0,0,0,0,1
2430,Pokémon Legends: Arceus,2022-01-28,13000,1800,3700,2600,"[Nintendo, Game Freak]",3.7,[Nintendo Switch],The Pokémon Legends: Arceus game honors past P...,...,0,0,0,0,0,0,0,0,0,1
12568,Persona 5,2016-09-15,20000,747,3500,2000,"[Deep Silver, P Studio]",4.1,"[PlayStation 4, PlayStation 3]","Persona 5, a turn-based JRPG with visual novel...",...,0,0,0,0,0,0,0,0,0,1


In [11]:
data_processed_func = encode_api_data(game_data, links_id, api_data)

/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column]=pd.to_datetime(df[column])
/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_2_features.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupe['gen'] = None
/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

### MinMax Scaled Data

In [13]:
X_train, y_train = make_training_data(data_processed_func)

In [14]:
X_train

,0,1,2,3,4,5,7,8,9,10,...,39,40,41,42,43,44,45,46,47,48
0,0.000130,0.000161,0.000167,0.000313,0.000263,0.000690,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.000174,0.000161,0.000417,0.000156,0.000000,0.000517,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.000174,0.000161,0.000417,0.000156,0.000000,0.000517,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.000196,0.000161,0.004000,0.013281,0.000526,0.005517,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.000043,0.000161,0.000000,0.001406,0.000000,0.000172,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35620,0.000065,0.000161,0.000167,0.001250,0.000000,0.000690,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
35621,0.000109,0.000161,0.000000,0.000156,0.000263,0.000345,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35622,0.000043,0.000161,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35623,0.000370,0.000161,0.000000,0.000000,0.000526,0.001207,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Yeo Johnson scaled data

In [15]:
X_train_yeo, y_train_yeo = make_training_data_box_yeo(data_processed_func)

## Predicting on MinMax

In [17]:
knn_model_minmax = KNeighborsRegressor(n_neighbors=11).fit(X_train, y_train)

In [31]:
zelda_mask = data_processed_func['title'] == "The Legend of Zelda: Breath of the Wild"
wipout_mask = data_processed_func['title'] == "Wipeout"
stardew_mask = data_processed_func['title'] == "Stardew Valley"

In [45]:
zelda = X_train[11399:11400]
wipeout = X_train[31357:31358]
stardew = X_train[13221:13222]


In [49]:
ind_list = list(knn_model_minmax.kneighbors(wipeout,n_neighbors=11)[1][0])

In [50]:
ind_list

[31357, 25559, 27658, 26112, 31001, 29597, 25177, 22788, 18555, 23001, 29745]

In [52]:
data_processed_func.columns

Index(['title', 'release_date', 'plays', 'playing', 'backlogs', 'wishlist',
       'developers', 'avg_review', 'platforms', 'description', 'total_reviews',
       'total_lists', 'main', 'ratings_zero_five', 'ratings_one_zero',
       'ratings_one_five', 'ratings_two_zero', 'ratings_two_five',
       'ratings_three_zero', 'ratings_three_five', 'ratings_four_zero',
       'ratings_four_five', 'ratings_five_zero', 'image', 'url', 'Adventure',
       'Indie', 'RPG', 'Simulator', 'Puzzle', 'Shooter', 'Strategy',
       'Platform', 'Arcade', 'Sport', 'gen_1', 'gen_2', 'gen_3',
       'Battle Royale', 'Co-operative', 'Massively Multiplayer Online (MMO)',
       'Multiplayer', 'Single player', 'Split screen', 'Action', 'Fantasy',
       'Science fiction', 'Comedy', 'Horror', 'Historical', 'Mystery',
       'Open world', 'Kids', 'Survival', 'Sandbox', 'Warfare', 'Party',
       'Drama', 'Educational', 'Erotic', 'Auditory', 'Bird view / Isometric',
       'First person', 'Side view', 'Text', 'Th

# Everything below was working on the above

In [7]:
reference['url'] = reference['url'].apply(remove_link_trail)

0        /games/darling-duality-winter-wish/
1                         /games/anger-foot/
2                 /games/life-is-feudal-mmo/
3             /games/the-making-of-karateka/
4                     /games/enigma-of-fear/
                        ...                 
36815                   /games/final-crisis/
36816                    /games/level-up--1/
36817                  /games/sea-battle--2/
36818              /games/we-are-illuminati/
36819                     /games/karate-cat/
Name: url, Length: 36820, dtype: object

In [8]:
data_with_id = reference.merge(links_id, left_on='url', right_on='link')

## Merging data_with_id to api data

In [10]:
all_data = data_with_id.merge(api_data, left_on='game_id', right_on='id')

In [ ]:
relevant_columns = all_data.drop(api_cols_to_drop, axis=1)

## Cetegorically encoding the api columns

In [14]:
prepped_data = prepare_json_df(all_data)

In [40]:
nec_data = prepped_data.drop(api_cols_to_drop, axis=1)
filled_data = nec_data.fillna('')

In [60]:
ohe_modes, mlb = categorical_encoder(filled_data['game_modes'])
ohe_theme= keeping_ohe_columns_and_dropping_the_originals(filled_data[['themes']], 16)
ohe_perspect, mlb = categorical_encoder(filled_data['player_perspectives'])

In [61]:
ohe_colums = pd.concat((ohe_modes, ohe_theme, ohe_perspect), axis=1)

In [67]:
pd.concat((filled_data, ohe_colums), axis=1)

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,platforms,description,...,Drama,Educational,Erotic,Auditory,Bird view / Isometric,First person,Side view,Text,Third person,Virtual Reality
0,Darling Duality: Winter Wish,2024-12-31,7,0,2,2,[Melancholy Marionette],2.2,"[Windows PC, Mac, Linux]",We all have multiple sides. Pick your poison! ...,...,0,0,0,0,0,0,0,1,0,0
1,Life Is Feudal: MMO,2023-12-31,9,0,5,1,[Long Tale Games],2.4,[Windows PC],Life is Feudal: MMO is an open-world survival ...,...,0,0,0,0,0,1,0,0,1,0
2,Life Is Feudal: MMO,2023-12-31,9,0,5,1,[Long Tale Games],2.4,[Windows PC],Life is Feudal: MMO is an open-world survival ...,...,0,0,0,0,0,1,0,0,1,0
3,They Speak From the Abyss,2023-12-31,10,0,48,85,"[2 Left Thumbs, Nikki Kalpa]",3.2,[Windows PC],Venture into the Abyss and face strange and ho...,...,0,0,0,0,0,1,0,0,0,0
4,Shred and Tear: Explosive Kajun,2023-12-31,3,0,0,9,"[Shady Corner Games, Lucid Realm Games]",1.5,[Windows PC],"An ultraviolent, fast-paced, sexy and gory Hac...",...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35620,Davigo,2023-12-31,4,0,2,8,[Davigo Studio],3.5,"[SteamVR, Oculus Quest 2]","DAVIGO is a VR vs. PC ""cross-reality"" battle g...",...,0,0,0,0,0,0,0,0,1,1
35621,Level Up!,2009-11-25,6,0,0,1,[],2.7,[Web browser],"Level Up! is a Game created by Titch007uk, and...",...,0,0,0,0,0,0,0,0,0,0
35622,Sea Battle,2014-02-07,3,0,0,0,[BYRIL],2.2,"[Android, iOS]",Sea Battle is a board game everybody loves sin...,...,0,0,0,0,0,0,0,0,0,0
35623,We Are Illuminati,2017-11-02,18,0,0,0,[Tapps Games],2.0,"[Android, iOS]","Control minds, insert subliminal messages into...",...,0,0,0,0,0,0,0,0,0,0
